In [1]:
from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic)

@register_line_magic
def lmagic(line):
    "my line magic"
    return line

@register_cell_magic
def cmagic(line, cell):
    "my cell magic"
    return line, cell

@register_line_cell_magic
def lcmagic(line, cell=None):
    "Magic that works both as %lcmagic and as %%lcmagic"
    if cell is None:
        print("Called as line magic")
        return line
    else:
        print("Called as cell magic")
        return line, cell

# In an interactive session, we need to delete these to avoid
# name conflicts for automagic to work on line magics.
del lmagic, lcmagic

In [9]:
%%cmagic
'abc'
'def'

('', "'abc'\n'def'\n")

In [11]:
%lmagic
'abc'
'def'

'def'

In [12]:
%%lcmagic
'abc'
'def'

Called as cell magic


('', "'abc'\n'def'\n")

In [160]:
import os
import subprocess
import argparse
from subprocess import CalledProcessError
from IPython.core.magic import (Magics, magics_class, cell_magic)
from IPython.core.magic_arguments import argument, magic_arguments, parse_argstring

# idea is this:
# for each cell with jupyda cell magic will be saved as a file in project folder
# the project will be compiled
# and run by main() function
# this is where nvcc lies
compiler = 'gcc'

def print_byte_stream(bstream):
    for line in bstream.decode().split('\n'):
        print(line)


@magics_class
class Jupyda(Magics):
    def __init__(self, shell):
        super(Jupyda, self).__init__(shell)
        
        # current working directory
        self.cwd = os.getcwd()
        
        # this is the project folder to save all the source file
        self.src = os.path.join(self.cwd, 'src')
        
        # create the folder
        if not os.path.exists(self.src):
            os.mkdir(self.src)
        
        # we will use a common file name for all the compiled files (like a.out)
        # self.aout = os.path.join(self.cwd, 'aout.o')
        self.aout = r'aout.o'
        
        
    @staticmethod
    def compile(src, aout, filepath):
        # compile it
        # subprocess.check_output([compiler, '-v', '-Wall', '-I='+src, '-o '+aout, filepath]
        #    , stderr=subprocess.STDOUT
        # )

        subprocess.check_output([compiler, '-Wall', '-o '+aout, filepath]
            #, shell=True
            , stderr=subprocess.STDOUT
        )
        
    @staticmethod
    def exec(obj):
        output = subprocess.check_output(
                ['./'+obj]
                , shell=True
                , stderr=subprocess.STDOUT)
        return output
        
    
    @magic_arguments()
    @argument('-n', '--name', type=str, help='entire cell content will be saved in file with this name.')
    # @argument('-c', '--compile', type=bool, help='Should be compiled?')
    @cell_magic
    def nvcc(self, line, cell=None):
        # parser for the arguments
        args = parse_argstring(self.nvcc, line)
        filename = args.name

        # write the cell content in a file in our project folder
        filepath = os.path.join(self.src, filename)
        with open(filepath, "w") as f:
            f.write(cell)
        f.close()
        
        output = b""
        try:
            # compile the file
            self.compile(self.src, self.aout, filepath)
            print("======== Compilation successful! ========")
            # run the file
            output = self.exec(self.aout)
        except CalledProcessError as cpe:
            print_byte_stream(cpe.output)
            
        print_byte_stream(output)

            
# register the magics function
ip = get_ipython()
ip.register_magics(Jupyda)

In [162]:
%%nvcc -n one.c

#include<stdio.h>œ
int foo(void) {
    printf("Hello World!!!\n");
    return 0;
}

Undefined symbols for architecture x86_64:
  "_main", referenced from:
     implicit entry/start for main executable
ld: symbol(s) not found for architecture x86_64
clang: error: linker command failed with exit code 1 (use -v to see invocation)


